# Transform Data with Spark in Azure Synapse Analytics

Spark allows you to work with and manipulate data into information

In this notebook, you'll consolidate heterogenous sources and create a homogenous output for consumption into a partitioned parquet format. This file can be used for consumption by a data scientist or data analyst for further analysis.

> **Note**: This notebook is designed to be run in an Azure Synapse Analytics Spark pool.


## Attach this notebook to a Spark pool

To run the code in this notebook, you'll need to use a Spark pool; so at the top of this notebook, in the **Attach to** list, select your Spark pool that was created in the setup.ps1 script.

## Explore Data

Before training a model, a data engineer will explore the data to ensure that its profile matches what is expected, which is usually in the form of a technical specification. The use of notebooks though, allows data professionals to place these specifications within the notebook itself and allows for much greater collaboration throughout the organization.

In this example, you'll explore some historical flight data with which we'll later transform into a denormalized structure and store it on disk for consumption by other data professionals downstream.


### Load Data Using an Explicit Schema

Let's start by loading some historical Sales Order data into a dataframe. If the structure of the data is known ahead of time, you can explicitly specify the schema for the dataframe.

Review the code in the cell below, which defines a schema for Sales Order data before loading it from all of the csv files within the data directoyr. Then click the **&#9655;** button to the left of the cell to run it.

> **Note**: The first time you run a cell in a notebook, the Spark pool must be started; which can take several minutes.

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

OrderSchema = StructType([
  StructField("SalesOrderNumber", StringType(), False),
  StructField("SalesOrderLineNumber", IntegerType(), False),
  StructField("OrderDate", DateType(), False),
  StructField("CustomerName", StringType(), False),
  StructField("EmailAddress", StringType(), False),
  StructField("Item", StringType(), False),
  StructField("Quantity", IntegerType(), False),
  StructField("UnitPrice", StringType(), False),
  StructField("TaxAmount", StringType(), False)
])

#let's populate the dataset using the schema noting that we're using the wildcard character 
#to grab all three of the csv files in the 'data' folder.
OrderDetails = spark.read.csv('/data/*.csv', schema=OrderSchema, header=True)
print("Data and schema loaded into OrderDetails successfully!")

In [ ]:
display(OrderDetails.limit(20))

### Infer a Data Schema
If the structure of the data source is unknown, you can have Spark automatically infer the schema.

In this case, you will load data about *Orders* without knowing the schema.

Run the following cell to load airport data from a text file, inferring the column names and data types automatically.

> **Note**: You will want to ensure that schemas are the same when using a wildcard character to prevent troubleshooting errors.

In [ ]:
OrderDetails2019 = spark.read.csv('/data/*.csv', header=True, inferSchema=True)
display(OrderDetails2019.limit(5))

### Count the Rows in a Dataframes
Now that you're familiar with working with dataframes, a key task when building predictive solutions is to explore the data, determing statistics that will help you understand the data before building predictive models. For example, how many rows of flight data do you actually have?

In [ ]:
# This spans across all the files to perform a count within the DataFrame
OrderDetails.count()

### Count inferred rows
This will execute the same query on inferred data from all of the files with a csv extension in the noted folder. The count should be the same as that above.

In [ ]:
# This code spans the single file pulled into the DataFrame
OrderDetails2019.count()

### Using a where clause to filter data
The where clause can be applied to dataframes in PySpark and are an effective method to separate data into sub-groups, such as Order data as shown below.

In [ ]:
# The first filter is by email address and performs a count.
print('Total rows in DataFrame where \
EmailAddress = morgan30@adventure-works.com with where clause')
print(OrderDetails.where(OrderDetails.EmailAddress == 'morgan30@adventure-works.com').count())
  
# The second uses the .show() function to list the results of the filtered dataframe query
print('They are  ')
OrderDetails.where(OrderDetails.EmailAddress == 'morgan30@adventure-works.com').show()

Using the same approach with a different column. 
You can view the show() function and other DataFrame functions at the [Apache Spark API Reference website](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.DataFrame.show.html).

In [ ]:
print('Total rows in DataFrame where SalesOrderNumber = SO43705 with where clause')
print(OrderDetails.where(OrderDetails.SalesOrderNumber == 'SO43705').count())
  
print('They are  ')
OrderDetails.where(OrderDetails.SalesOrderNumber == 'SO45347').show()

### Use Dataframe Methods
Spark DataFrames provide functions that you can use to extract and manipulate data. For example, you can use the **select** function to return a new dataframe containing columns selected from an existing dataframe.

In [ ]:
# the use of DataFrames from another DataFrames is a quick way to create working tables
# you will want to manage these as you go along but they allow the data engineer to try different analysis methods easily.
OrderDetailSQL = OrderDetails.select("CustomerName", "OrderDate", "SalesOrderNumber", "OrderDate", "Item", "Quantity", "UnitPrice", "TaxAmount")
display(OrderDetailSQL.limit(20))

### Create a Calendar table in the 
The next few steps will use the ***explode, sequence, and to_date functions*** to create a structure for populating a dim_calendar.

In [ ]:
from pyspark.sql.functions import explode, sequence, to_date
#set parameters of the dates to use
startDate = '2010-01-01'
endDate = '2025-12-31'

(
  spark.sql(f"select explode(sequence(to_date('{startDate}'), to_date('{endDate}'), interval 1 day)) as calendarDate")
    .createOrReplaceTempView('calendar')
)

### Expand on calendar view
Let's add some more insight into the calendar table for encoding the Orders data further down to include Fiscal Year and Quarter

In [ ]:
spark.sql(f"""select year(calendarDate) * 10000 + month(calendarDate) * 100 + day(calendarDate) as dateid,
  CalendarDate,
  year(calendarDate) AS CalendarYear,
  date_format(calendarDate, 'MMMM') as CalendarMonth,
  month(calendarDate) as MonthOfYear,
  date_format(calendarDate, 'EEEE') as CalendarDay,
  dayofweek(calendarDate) AS DayOfWeek,
  weekday(calendarDate) + 1 as DayOfWeekStartMonday,
  case
    when weekday(calendarDate) < 5 then 'Y'
    else 'N'
  end as IsWeekDay,
  dayofmonth(calendarDate) as DayOfMonth,
  case
    when calendarDate = last_day(calendarDate) then 'Y'
    else 'N'
  end as IsLastDayOfMonth,
  dayofyear(calendarDate) as DayOfYear,
  weekofyear(calendarDate) as WeekOfYearIso,
  quarter(calendarDate) as QuarterOfYear,
  /* Use fiscal periods needed by organization fiscal calendar */
  case
    when month(calendarDate) >= 7 then year(calendarDate) + 1
    else year(calendarDate)
  end as FiscalYear,
  (month(calendarDate) + 5) % 12 + 1 AS FiscalMonth
from
  calendar
order by
  calendarDate""").createOrReplaceTempView('dim_Calendar')

#### Query the table with a SELECT statement using magic key to first define the language. 
The current languages supported in Synapse Spark notebooks include: 
- PySpark (Python)
- Spark (Scala)
- Spark SQL
- .NET Spark (C#)
- SparkR (R) - In Preview

In [ ]:

%%sql
SELECT * from dim_Calendar LIMIT 20

### Split Customer Name into more searchable format
Splitting the customer first name and last name is a common need in data transformation. It allows for easier searching. The following code will bring the OrderDetails dataframe as previously defined and add teh columns FirstName and LastName to the end which were split from teh CustomerName column.

In [ ]:
#Using the code above, let's create a new sql view/table
### String Split of the column in pyspark
from pyspark.sql.functions import split
 
temp_df = OrderDetails.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))
temp_df.createOrReplaceTempView("SQLOrderDetails")

### Enocde the Calendar Information ###
Now we can join the two temporary views together which split the customer name into two columns and join to the **dim_calendar** table 
to encode the data with a ***smart*** key and also brings in ***FiscalYear*** and ***FiscalMonth*** into the result set. We also create 
a new column named **OrderTotal** to be stored on disk with the other data.

In [ ]:
#Create the temporary dataframe using a spark.sql query which joins the two temporary tables and returns transformed and encoded data.
temp_df = spark.sql(f"""SELECT SalesOrderNumber, SalesOrderLineNumber, b.CalendarDate, b.FiscalYear, b.FiscalMonth, LastName as CustomerLastName, FirstName as CustomerFirstName, EmailAddress Item, Quantity, UnitPrice, TaxAmount, UnitPrice * Quantity + TaxAmount as OrderTotal FROM SQLOrderDetails a INNER JOIN dim_Calendar b
ON a.OrderDate=b.CalendarDate ORDER BY OrderDate, CustomerName""")

### Partition the data by OrderDate and CustomerName
The following code will create a set of files that are partitioned by OrderDate and CustomerName and store it in a parquet file format which is stored in a distributed fashion for higher compression of the files and for performance when working with the data in a distributed file system.

In [ ]:

#write the partitioned dataframe
temp_df.write.partitionBy("FiscalYear","FiscalMonth").mode("overwrite").parquet("/data/OrdersTransform")

### Query the newly created parquet file that is now in our Azure Data Lake
we can now pull in the partitioned parquet file for consumption by other teams.

In [ ]:
# read content of file
df = spark.read.parquet('/data/OrdersTransform')
df.show(10)

### Filter based on the partitioned columns


In this notebook, you've explored how to use a spark notebook to query data files within the datalake to perform some basic analysis with pyspark and pysql. You then exported those results into a format named parquet which is optimized for distributed and massively parrallel processsing (MPP) systems which are Analytical workloads. The value-add of parquet is that it is optimized for these workloads by its column-oriented structure and compression which can improve performance and reduce overall storage and compute costs.

We've only scratched the surface of the power of notebooks. To learn more, see the [Apache Spark Notebooks Documentation](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-development-using-notebooks).